<a href="https://colab.research.google.com/github/victoriamazilu/human-detection/blob/main/human_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [ ]:
!nvidia-smi

Mon Jun  2 22:26:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   70C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


## Install YOLOv8

YOLOv8 can be installed in two ways - from the source and via pip. This is because it is the first iteration of YOLO to have an official package.

In [ ]:
# Pip install method (recommended)
!pip install numpy==1.24.3 --quiet

!pip install ultralytics==8.2.103 -q

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.103 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.4/112.6 GB disk)


In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:
model = YOLO(f'{HOME}/yolov8s-seg.pt')
#results = model.predict(source='https://media.roboflow.com/notebooks/examples/dog.jpeg', conf=0.25)

100%|██████████| 22.8M/22.8M [00:00<00:00, 60.5MB/s]


In [ ]:
%%html
<video id="video" autoplay playsinline style="display:none;"></video>
<canvas id="canvas" style="display:none;"></canvas>
<script>
(async () => {
  const video  = document.getElementById('video');
  const canvas = document.getElementById('canvas');
  const ctx    = canvas.getContext('2d');

  try {
    const stream = await navigator.mediaDevices.getUserMedia({video:true});
    video.srcObject = stream;
    await video.play();
  } catch (e) {
    console.error('Camera access denied or not available', e);
    return;
  }

  canvas.width  = video.videoWidth;
  canvas.height = video.videoHeight;
  google.colab.output.setIframeHeight(canvas.height + 20);

  window.captureFrame = () => {
    ctx.drawImage(video, 0, 0, canvas.width, canvas.height);
    return canvas.toDataURL('image/jpeg', 0.8);
  };
</script>


# First Day solution

In [ ]:
# ───── Combined JS + Widget + Segmentation Loop ─────
from IPython.display import Javascript, display
from google.colab.output import eval_js
from base64 import b64decode
import cv2, numpy as np, time
import ipywidgets as widgets
from ultralytics import YOLO

# 1) Inject the JS into the notebook DOM
display(Javascript("""
(async () => {
  const v = document.createElement('video');
  v.autoplay = v.playsInline = true;
  v.style.display = 'none';
  document.body.appendChild(v);

  const c = document.createElement('canvas');
  c.style.display = 'none';
  document.body.appendChild(c);
  const ctx = c.getContext('2d');

  const stream = await navigator.mediaDevices.getUserMedia({video:true});
  v.srcObject = stream;
  await v.play();

  c.width = v.videoWidth;
  c.height = v.videoHeight;
  google.colab.output.setIframeHeight(c.height + 20);

  window.captureFrame = () => {
    ctx.drawImage(v, 0, 0, c.width, c.height);
    return c.toDataURL('image/jpeg', 0.8);
  };
})();
"""))

# 2) Wait for captureFrame() to become available
print("⏳ Waiting for camera…")
for _ in range(25):
    try:
        if eval_js("typeof captureFrame") == 'function':
            print("✅ Camera ready!")
            break
    except Exception:
        pass
    time.sleep(0.2)
else:
    raise RuntimeError("camera never initialized – did you allow permission?")

# 3) Helpers to pull & decode frames
def get_frame():
    data = eval_js('captureFrame()')
    _, b64 = data.split(',', 1)
    img = cv2.imdecode(
        np.frombuffer(b64decode(b64), np.uint8),
        cv2.IMREAD_COLOR
    )
    return img

# 4) Load your trained model (adjust path if necessary)
model = YOLO('/content/yolov8s-seg.pt')
person_class = 0  # usually 0 for COCO “person”

# 5) Create a persistent widget & show once
img_widget = widgets.Image(format='jpeg')
display(img_widget)

# 6) Run the live loop, updating widget in-place
print("▶️ Starting live person segmentation. Interrupt (⏹) to stop.")
try:
    while True:
        frame = get_frame()
        res   = model.predict(frame, classes=[person_class], stream=False)
        vis   = res[0].plot()
        _, jpg = cv2.imencode('.jpg', vis)
        img_widget.value = jpg.tobytes()
        # tiny sleep so you don’t hammer eval_js
        time.sleep(0.02)
except KeyboardInterrupt:
    print("⏹ Segmentation stopped.")


<IPython.core.display.Javascript object>

⏳ Waiting for camera…


RuntimeError: camera never initialized – did you allow permission?